<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/Project/Chatbot/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install requests beautifulsoup4

from bs4 import BeautifulSoup
from nltk.chat.util import Chat, reflections
from firebase_admin import credentials
from firebase_admin import firestore
import firebase_admin
import nltk
import requests

# activate only if run this file directly and not from searchEngine
# !git clone "https://github.com/ndvp39/CloudComputing-tirgul.git"
%cd /content/CloudComputing-tirgul/Project/Index/
%run index.ipynb

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
def getLinkFromString(w,linkID):
  for word, links in index.items(): # should be from database directly and not from database , index = final_index
    if(word == w):
      return list(links.keys())[linkID]

In [ ]:
def get_first_paragraph_from_link(url):
    try:
        # Make an HTTP request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the first <p> tag with the specified ID
            h2_tag = soup.find('h2', {'id': 'overview'})
            if h2_tag:

              para = h2_tag.find_next('p')
              if para:
                return para.get_text()
              else:
                return "Paragraph after h2 with id: overview not found."

            else:

              div_tags = soup.find_all('div', {'class': 'rh-generic--component'})
              # Check if there is at least a second occurrence
              if len(div_tags) >= 2:
                second_div_tag = div_tags[1]  # Select the second occurrence
                para = second_div_tag.find_next('p').get_text()
              elif len(div_tags) == 1:
                first_div_tag = div_tags[0]  # Select the first occurrence
                para = first_div_tag.find_next('p').get_text()
              else:
                return ""

            return para



        else:
            return f"Failed to fetch the page. Status code: {response.status_code}"

    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
cloud_explanation = """Private Cloud: A private cloud is a cloud computing environment that is dedicated solely to one organization.
It is usually managed either by the organization itself or by a third-party service provider. The infrastructure and services are not shared
with other organizations, offering greater control, security, and customization options. Private clouds are typically used by large enterprises
or organizations with specific compliance or security requirements.
Public Cloud: A public cloud is a cloud computing environment that is hosted and managed by a third-party service provider and made available
to multiple organizations or individuals over the internet. The infrastructure and resources are shared among multiple users,
 offering scalability, flexibility, and cost-effectiveness. Public clouds are accessible to anyone who wants to use them and are often used
 for general-purpose computing needs such as web hosting, development, and testing.
Hybrid Cloud: A hybrid cloud is a computing environment that combines elements of both private and public clouds. It allows organizations
 to leverage the benefits of both environments by integrating on-premises infrastructure with public cloud resources. This integration enables
 data and applications to be dynamically moved between the two environments based on changing workload demands, security requirements, and cost
 considerations. Hybrid clouds offer greater flexibility, scalability, and customization options compared to using either private or public clouds alone.
 They are commonly used by organizations that want to retain control over sensitive data while taking advantage of the scalability and cost-effectiveness of
 public cloud services."""

In [ ]:
def patterns_to_dict(patterns):
    patterns_dict = []
    for pattern, responses in patterns:
        pattern_dict = {
            'pattern': pattern,
            'responses': responses
        }
        patterns_dict.append(pattern_dict)
    return patterns_dict

In [ ]:
def create_patterns():
  for word, links in index.items(): # take the index from DB , index = final_index
    answers = []
    for i in range(len(links)):
      link = getLinkFromString(word,i)
      first_para = get_first_paragraph_from_link(link)
      if first_para == "": # also need to remove dates
        continue
      answers.append(first_para + f"you can read more in the link: {link}")
      print(first_para)
    to_append = ( r'(.*)' + word + r'(.*)', answers)
    patterns.append(to_append)

In [ ]:
def upload_to_db(data_list, collection_name):
  try:
    for data in data_list:
        doc_ref = db.collection(collection_name).add(data)
    print("All documents uploaded successfully.")
  except Exception as e:
      print("Error:", e)

In [ ]:
def get_patterns_from_db(db_collection_name):
    patterns = []
    collection_ref = db.collection(db_collection_name)
    docs = collection_ref.get()
    for doc in docs:
        data = doc.to_dict()
        pattern = r'(.*)' + data['pattern'] + r'(.*)'
        responses = data['responses']
        patterns.append((pattern, responses))
    return patterns


In [ ]:
# Define some patterns and responses. ______________ take this patterns fron database ______________
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me PantherBOT.', 'I go by the name PantherBOT.']),
    (r'your name (.*)', ['You can call me PantherBOT.', 'I go by the name PantherBOT.']),
    (r'give me good cloud website', ['Sure, here: www.redhat.com', 'ofCourse: you can go to amazon, which works with RedHat website check both']),
    (r'What kinds of clouds are there ?', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(.*)kinds of clouds(.*)?(.*)', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'please explain further about clouds(.*)', ['cloud_explanation', "for the moment I too tired, may be later I will explain. You should've listen in class "])
    # You can add more patterns and responses here
]

db_collection_name = "ChatBotPatterns"
cred = credentials.Certificate("/content/CloudComputing-tirgul/Project/Json/sak.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred)
db = firestore.client()

##### The long functions -> do not activate unless you want to change bot dialog ######
# index = get_index_from_db()
# create_patterns()
final_patterns = patterns_to_dict(patterns)

###### Uploading to DB ######
# upload_to_db(final_patterns, db_collection_name)

# print what goes inside the db
# print(final_patterns)
# print(get_patterns_from_db(db_collection_name))
